## Problem statement

### How good is one batter against CSK at Chepauk Stadium when they're playing 3 spinners?

1. Apart from career statistics (matches/runs/avg/sr),what information can be derived from a player’s historic data?
2. Strengths & Weaknesses against Spinners in middle overs(Mostly in T20's spinners bowled in middle overs i.e, 7-15 Overs).
3. His contribution to team’s win
4. Performance against Chennai Super Kings (CSK).
5. Performance in Chepauk Statdium , Chennai.

### Batting Performance Analysis

1. Player Analysis
    -  Player-level statistics --- (runs/sr/RPI/BPB)
    -  Type of player --- (anchor/strike-rotator/hitter)
    -  Performance in diff phases of play --- (PP/Middle/Death)
    -  Performance by innings --- (Bat 1st/2nd)
2. Team Analysis
    - Performance against oppositions --- (RPI/SR)
3. Venue Analysis
    - Performance in venues
    - Venue type --- (runrates, wickets, ..)

In [2]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
os.chdir("C:\\Users\\SWARNASASH\\Downloads\\Desktop Material\\IPL DATA(2008-24)")

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
deliveries = pd.read_csv("detailed_deliveries.csv")
deliveries.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,is_dot,is_one,is_two,is_three,is_four,is_six,Over,match_phase,is_bowled,is_caught,is_lbw,is_stumped,year
0,335982,2007/08,18-04-2008,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,0,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,1,powerplay,0,0,0,0,2008


In [4]:
deliveries.shape

(260920, 35)

In [5]:
deliveries['season'] = deliveries['season'].apply(str)
deliveries['season'].unique()

array(['2007/08', '2009', '2009/10', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2019', '2018', '2020/21', '2021', '2022',
       '2023', '2024'], dtype=object)

In [6]:
def batter_performance(df, innings = None, match_phase = None, batting_team = None, bowling_team = None, venue = None, bowler = None, season = None):

    if innings != None:
        df = df[df["innings"] == innings]

    if match_phase != None:
        df = df[df["match_phase"] == match_phase]
        
    if batting_team != None:
        if isinstance(batting_team, list):
            df = df[df["batting_team"].isin(batting_team)]
        else:
            df = df[df["batting_team"] == batting_team]

    if bowling_team != None:
        if isinstance(bowling_team, list):
            df = df[df["bowling_team"].isin(bowling_team)]
        else:
            df = df[df["bowling_team"] == bowling_team]

    if venue != None:
        if isinstance(venue, list):
            df = df[df["venue"].isin(venue)]
        else:
            df = df[df["venue"] == venue]

    if bowler != None:
        if isinstance(bowler, list):
            df = df[df["bowler"].isin(bowler)]
        else:
            df = df[df["bowler"] == bowler]

    if season != None:
        if isinstance(season, list):
            df = df[df["season"].isin(season)]
        else:
            df = df[df["season"] == season]

    def batting_average(runs, dismissals):
        if dismissals > 0:
            return np.round((runs/dismissals), 2)
        else :
            return runs/1

    def balls_per_dismissal(balls, dismissals):
        if dismissals > 0:
            return np.round((balls/dismissals), 2)
        else :
            return balls/1
    
    def balls_per_boundary(balls, boundaries):
        if boundaries > 0:
            return np.round((balls/boundaries), 2)
        else:
            return balls/1

    def dot_percentage(dots, balls):
        if balls > 0:
            return np.round(100*(dots/balls), 2)
        else:
            return 0

    runs = pd.DataFrame(df.groupby(["striker"])["runs_off_bat"].sum()).reset_index().rename(columns={"striker":"batter","runs_off_bat":"runs"})
    innings = pd.DataFrame(df.groupby(["striker"])["match_id"].apply(lambda x : len(list(np.unique(x))))).reset_index().rename(columns={"striker":"batter","match_id":"innings"})
    balls = pd.DataFrame(df.groupby(["striker"])["match_id"].count()).reset_index().rename(columns={"striker":"batter","match_id":"balls"})
    dismissals = pd.DataFrame(df.groupby(["striker"])["player_dismissed"].count()).reset_index().rename(columns={"striker":"batter","player_dismissed":"dismissals"})
    
    dots = pd.DataFrame(df.groupby(["striker"])["is_dot"].sum()).reset_index().rename(columns={"striker":"batter","is_dot":"dots"})
    ones = pd.DataFrame(df.groupby(["striker"])["is_one"].sum()).reset_index().rename(columns={"striker":"batter","is_one":"ones"})
    twos = pd.DataFrame(df.groupby(["striker"])["is_two"].sum()).reset_index().rename(columns={"striker":"batter","is_two":"twos"})
    threes = pd.DataFrame(df.groupby(["striker"])["is_three"].sum()).reset_index().rename(columns={"striker":"batter","is_three":"threes"})
    fours = pd.DataFrame(df.groupby(["striker"])["is_four"].sum()).reset_index().rename(columns={"striker":"batter","is_four":"fours"})
    sixes = pd.DataFrame(df.groupby(["striker"])["is_six"].sum()).reset_index().rename(columns={"striker":"batter","is_six":"sixes"})
    
    df = innings.merge(runs, on= "batter").merge(balls, on= "batter").merge(dismissals, on= "batter").merge(dots, on= "batter").merge(ones, on= "batter").merge(twos, on= "batter").merge(threes, on= "batter").merge(fours, on= "batter").merge(sixes, on= "batter")
    
    df["strike_rate"] = df.apply(lambda x: np.round (100 * (x["runs"]/x["balls"]),2), axis = 1)
    df["batting_average"] = df.apply(lambda x: batting_average(x["runs"], x["dismissals"]), axis = 1)
    df["runs_per_inning"] = df.apply(lambda x: np.round (x["runs"]/x["innings"],2), axis = 1)
    df["balls_per_dismissal"] = df.apply(lambda x: balls_per_dismissal(x["balls"], x["dismissals"]), axis = 1)
    df["balls_per_boundary"] = df.apply(lambda x: balls_per_boundary(x["balls"],(x["fours"]+x["sixes"])), axis = 1)  
    df["dot_percentage"] = df.apply(lambda x: dot_percentage(x["dots"],x["balls"]), axis = 1) 

    return df

### Venue and Phase-Specific Insights:

- **Chepauk Stadium** favors spin bowlers, making it challenging for batters, especially in the middle overs. Successful batters often exhibit high adaptability to spin variations and can maintain both a steady strike rate and boundary rate despite the conditions.
- **Middle Overs (7–15)**: This phase is pivotal as it often dictates the scoring tempo for the innings. Batters who perform well in this phase usually have both strong spin-handling skills and a well-balanced approach to run accumulation and boundary-scoring.

## Target Batters:
- Batted against 3 spinners ( Mostly spinners bowl in the middle phase of the Game)
- Batting Against CSK
- Batting at Chepauk Stadium

### Middle_Over_Batting_Performance_Metrics_Assumptions

![Middle_Over_Batting_Performance_Metrics_Assumptions](Performance_Metrics\\Middle_Over_Batting_Performance_Metrics_Assumptions.png)


### Middle_Over_Battings_PairWise_Matrix

![Middle_Over_Battings_PairWise_Matrix](Performance_Metrics\\Middle_Over_Batting_Pairwise_Comparison_Matrix.png)


### Middle_Over_Battings_Criteria_Weights

![Middle_Over_Battings_Criteria_Weights](Performance_Metrics\\Middle_Over_Batting_Criteria_Weights.png)

- **(*)** indicate retired players.

## Calculation Of Performance Score

In [18]:
def batter_performance_score(target_data, match_phase, minimum_innings = 1, minimum_runs = 0, minimum_balls = 1):
    
    # Player should have played greater than minimum_innings
    
    target_data = target_data[target_data["innings"] >= minimum_innings]

    # Player should have scored greater than minimum_runs

    target_data = target_data[target_data["runs"] >= minimum_runs]

    # Player should have faced greater than minimum_runs

    target_data = target_data[target_data["balls"] >= minimum_balls]

    # criteria weights in different match_phase

    if match_phase == "powerplay" :
        wt_SR, wt_RPI, wt_BPD, wt_dot_percentage, wt_BPB = 0.26, 0.11, 0.07, 0.37, 0.19
    if match_phase == "middle" :
        wt_SR, wt_RPI, wt_BPD, wt_dot_percentage, wt_BPB = 0.12, 0.24, 0.16, 0.40, 0.08
    if match_phase == "death" :
        wt_SR, wt_RPI, wt_BPD, wt_dot_percentage, wt_BPB = 0.18, 0.11, 0.07, 0.36, 0.28
                    
    # Step1 : Normalizing all values to same dimensions
    
    # square of all the values
    
    target_data["cal_SR"] = target_data["strike_rate"].apply(lambda x: x*x)
    target_data["cal_RPI"] = target_data["runs_per_inning"].apply(lambda x: x*x)
    target_data["cal_BPD"] = target_data["balls_per_dismissal"].apply(lambda x: x*x)
    target_data["cal_dot_percentage"] = target_data["dot_percentage"].apply(lambda x: x*x)
    target_data["cal_BPB"] = target_data["balls_per_boundary"].apply(lambda x: x*x)
    
    # square root of the values
        
    sq_SR, sq_RPI, sq_BPD, sq_dot_percentage, sq_BPB = np.sqrt(target_data[["cal_SR","cal_RPI","cal_BPD","cal_dot_percentage","cal_BPB"]].sum(axis=0))
        
    # Divide by squared value
        
    target_data["cal_SR"] = target_data["cal_SR"].apply(lambda x: x/sq_SR)
    target_data["cal_RPI"] = target_data["cal_RPI"].apply(lambda x: x/sq_RPI)
    target_data["cal_BPD"] = target_data["cal_BPD"].apply(lambda x: x/sq_BPD)
    target_data["cal_dot_percentage"] = target_data["cal_dot_percentage"].apply(lambda x: x/sq_dot_percentage)
    target_data["cal_BPB"] = target_data["cal_BPB"].apply(lambda x: x/sq_BPB)
    
    # Multiply with criteria_weights
        
    target_data["cal_SR"] = target_data["cal_SR"].apply(lambda x: x*wt_SR)
    target_data["cal_RPI"] = target_data["cal_RPI"].apply(lambda x: x*wt_RPI)
    target_data["cal_BPD"] = target_data["cal_BPD"].apply(lambda x: x*wt_BPD)
    target_data["cal_dot_percentage"] = target_data["cal_dot_percentage"].apply(lambda x: x*wt_dot_percentage)
    target_data["cal_BPB"] = target_data["cal_BPB"].apply(lambda x: x*wt_BPB)
        
    # Calculate the best and worst valus
        
    best_SR, worst_SR = max(target_data["cal_SR"]), min(target_data["cal_SR"])
    best_RPI, worst_RPI = max(target_data["cal_RPI"]), min(target_data["cal_RPI"])
    best_BPD, worst_BPD = max(target_data["cal_BPD"]), min(target_data["cal_BPD"])
    best_dot_percentage, worst_dot_percentage = min(target_data["cal_dot_percentage"]), max(target_data["cal_dot_percentage"])
    best_BPB, worst_BPB = min(target_data["cal_BPB"]), max(target_data["cal_BPB"])

    #Step 2 : Comparision against the best & worst values
    
    # Calculate the best and worst deviated valus
    
    target_data["dev_best_SR"] = target_data["cal_SR"].apply(lambda x:(x-best_SR)*(x-best_SR))
    target_data["dev_best_RPI"] = target_data["cal_RPI"].apply(lambda x:(x-best_RPI)*(x-best_RPI))
    target_data["dev_best_BPD"] = target_data["cal_BPD"].apply(lambda x:(x-best_BPD)*(x-best_BPD))
    target_data["dev_best_dot_percentage"] = target_data["cal_dot_percentage"].apply(lambda x:(x-best_dot_percentage)*(x-best_dot_percentage))
    target_data["dev_best_BPB"] = target_data["cal_BPB"].apply(lambda x:(x-best_BPB)**2)
        
    target_data["dev_best_sqrt"] = target_data.apply(lambda x: x["dev_best_SR"] + x["dev_best_RPI"] + x["dev_best_BPD"] + x["dev_best_dot_percentage"] + x["dev_best_BPB"], axis =1 )
        
    target_data["dev_worst_SR"] = target_data["cal_SR"].apply(lambda x:(x-worst_SR)*(x-worst_SR))
    target_data["dev_worst_RPI"] = target_data["cal_RPI"].apply(lambda x:(x-worst_RPI)*(x-worst_RPI))
    target_data["dev_worst_BPD"] = target_data["cal_BPD"].apply(lambda x:(x-worst_BPD)*(x-worst_BPD))
    target_data["dev_worst_dot_percentage"] = target_data["cal_dot_percentage"].apply(lambda x:(x-worst_dot_percentage)*(x-worst_dot_percentage))
    target_data["dev_worst_BPB"] = target_data["cal_BPB"].apply(lambda x:(x-worst_BPB)*(x-worst_BPB))
        
    target_data["dev_worst_sqrt"] = target_data.apply(lambda x: x["dev_worst_SR"] + x["dev_worst_RPI"] + x["dev_worst_BPD"] + x["dev_worst_dot_percentage"] + x["dev_worst_BPB"], axis =1 )
    
    #Step 3 : Overall score for each player ( values from 0 - 1)
    
    target_data["score"] = target_data.apply(lambda x: np.round(x["dev_worst_sqrt"]/(x["dev_worst_sqrt"]+x["dev_best_sqrt"]),3),axis=1)

    target_data = target_data.reset_index()
    
    return target_data[["batter","innings","runs","balls","dismissals","fours","sixes","dot_percentage","strike_rate","batting_average","score"]]

## 1st Innings Performance

In [19]:
batter_info = batter_performance(deliveries,match_phase="middle",innings=1,bowling_team="Chennai Super Kings",venue="MA Chidambaram Stadium")
batter_info.head()

,batter,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,strike_rate,batting_average,runs_per_inning,balls_per_dismissal,balls_per_boundary,dot_percentage
0,A Ashish Reddy,1,1,2,0,1,1,0,0,0,0,50.0,1.0,1.0,2.0,2.0,50.00
1,A Mishra,1,14,16,1,9,4,1,0,2,0,87.5,14.0,14.0,16.0,8.0,56.25
2,A Raghuvanshi,1,0,1,1,1,0,0,0,0,0,0.0,0.0,0.0,1.0,1.0,100.00
3,A Symonds,1,36,32,0,13,14,1,0,2,2,112.5,36.0,36.0,32.0,8.0,40.62
4,AB Agarkar,1,7,4,0,1,1,1,0,1,0,175.0,7.0,7.0,4.0,4.0,25.00


In [20]:
batter_score = batter_performance_score(batter_info, match_phase= "middle", minimum_innings = 2, minimum_balls=25)
batter_score.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,AB de Villiers,2,45,31,1,6,0,29.03,145.16,45.0,0.898
1,AD Russell,2,56,47,0,3,4,44.68,119.15,56.0,0.453
2,AL Menaria,2,23,32,1,1,0,46.88,71.88,23.0,0.028
3,JC Buttler,2,38,35,1,0,2,25.71,108.57,38.0,0.725
4,KD Karthik,4,61,54,2,6,1,37.04,112.96,30.5,0.403


In [21]:
top_batter = batter_score.sort_values(by= ["score"], ascending= False).reset_index().drop(columns=["index"], axis=1)
top_batter.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,SR Watson,2,53,37,1,3,2,21.62,143.24,53.00,0.976
1,AB de Villiers,2,45,31,1,6,0,29.03,145.16,45.00,0.898
2,RG Sharma,2,50,34,1,4,3,35.29,147.06,50.00,0.824
3,JC Buttler,2,38,35,1,0,2,25.71,108.57,38.00,0.725
4,V Kohli,5,124,111,3,8,3,32.43,111.71,41.33,0.684


#### Top Batter:

    1. SR Watson* (Score : 0.976)
    2. AB de Villiers* (Score: 0.898)
    3. RG Sharma (Score : 0.824)

## 2nd Innings Performance

In [22]:
batter_info = batter_performance(deliveries,match_phase="middle",innings=2,bowling_team="Chennai Super Kings",venue="MA Chidambaram Stadium")
batter_info.head()

,batter,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,strike_rate,batting_average,runs_per_inning,balls_per_dismissal,balls_per_boundary,dot_percentage
0,A Badoni,1,10,8,0,1,4,3,0,0,0,125.00,10.0,10.00,8.0,8.0,12.50
1,A Mishra,1,8,9,0,4,4,0,0,1,0,88.89,8.0,8.00,9.0,9.0,44.44
2,A Mithun,1,11,8,1,3,3,0,0,2,0,137.50,11.0,11.00,8.0,4.0,37.50
3,AB de Villiers,3,61,48,1,15,20,6,1,5,1,127.08,61.0,20.33,48.0,8.0,31.25
4,AC Gilchrist,1,17,12,1,4,5,0,0,3,0,141.67,17.0,17.00,12.0,4.0,33.33


In [23]:
batter_score = batter_performance_score(batter_info, match_phase= "middle",minimum_innings=2, minimum_balls=25)
batter_score.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,AB de Villiers,3,61,48,1,5,1,31.25,127.08,61.0,0.419
1,AT Rayudu,3,22,31,1,0,0,32.26,70.97,22.0,0.199
2,CL White,2,44,44,1,5,0,36.36,100.00,44.0,0.263
3,DA Miller,4,54,47,3,5,1,34.04,114.89,18.0,0.251
4,DA Warner,3,77,77,2,1,3,36.36,100.00,38.5,0.249


In [24]:
top_batter = batter_score.sort_values(by= ["score"], ascending= False).reset_index().drop(columns=["index"], axis=1)
top_batter.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,SE Marsh,2,90,52,1,6,6,23.08,173.08,90.0,0.846
1,N Pooran,2,66,32,1,5,5,21.88,206.25,66.0,0.808
2,MP Stoinis,2,82,53,1,3,5,18.87,154.72,82.0,0.767
3,SR Watson,2,71,38,2,6,5,28.95,186.84,35.5,0.695
4,G Gambhir,3,92,78,1,5,2,29.49,117.95,92.0,0.630


#### Top Batter:

    1. SE Marsh* (Score : 0.846)
    2. MP Stoinis (Score: 0.808)
    3. N Pooran (Score : 0.767)

## Overall Performance

In [26]:
batter_info = batter_performance(deliveries,match_phase="middle",bowling_team="Chennai Super Kings",venue="MA Chidambaram Stadium")
batter_info.head()

,batter,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,strike_rate,batting_average,runs_per_inning,balls_per_dismissal,balls_per_boundary,dot_percentage
0,A Ashish Reddy,1,1,2,0,1,1,0,0,0,0,50.0,1.0,1.0,2.0,2.00,50.0
1,A Badoni,1,10,8,0,1,4,3,0,0,0,125.0,10.0,10.0,8.0,8.00,12.5
2,A Mishra,2,22,25,1,13,8,1,0,3,0,88.0,22.0,11.0,25.0,8.33,52.0
3,A Mithun,1,11,8,1,3,3,0,0,2,0,137.5,11.0,11.0,8.0,4.00,37.5
4,A Raghuvanshi,1,0,1,1,1,0,0,0,0,0,0.0,0.0,0.0,1.0,1.00,100.0


In [27]:
batter_score = batter_performance_score(batter_info, match_phase= "middle",minimum_innings=4, minimum_balls=50)
batter_score.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,AB de Villiers,5,106,79,2,11,1,30.38,134.18,53.00,0.688
1,AD Russell,4,74,60,1,5,4,40.00,123.33,74.00,0.595
2,DA Miller,5,73,62,4,7,1,33.87,117.74,18.25,0.344
3,DA Warner,4,109,100,3,2,4,32.00,109.00,36.33,0.475
4,IK Pathan,5,43,52,1,2,2,50.00,82.69,43.00,0.133


In [28]:
top_batter = batter_score.sort_values(by= ["score"], ascending= False).reset_index().drop(columns=["index"], axis=1)
top_batter.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,SR Watson,4,124,75,3,9,7,25.33,165.33,41.33,0.828
1,AB de Villiers,5,106,79,2,11,1,30.38,134.18,53.00,0.688
2,AD Russell,4,74,60,1,5,4,40.00,123.33,74.00,0.595
3,V Kohli,8,202,172,6,12,6,32.56,117.44,33.67,0.477
4,DA Warner,4,109,100,3,2,4,32.00,109.00,36.33,0.475


#### Top Batter:

    1. SR Watson* (Score : 0.828)
    2. AB de Villiers* (Score: 0.688)

## Recent Performance(Last 5 Seasons)

### 1st Innings

In [29]:
batter_info = batter_performance(deliveries,match_phase="middle", innings=1,bowling_team="Chennai Super Kings",venue="MA Chidambaram Stadium", season=['2020/21','2021','2022','2023','2024'])
batter_info.head()

,batter,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,strike_rate,batting_average,runs_per_inning,balls_per_dismissal,balls_per_boundary,dot_percentage
0,A Raghuvanshi,1,0,1,1,1,0,0,0,0,0,0.00,0.0,0.0,1.0,1.0,100.00
1,AK Markram,1,12,12,1,3,8,0,0,1,0,100.00,12.0,12.0,12.0,12.0,25.00
2,Abhishek Sharma,1,10,9,1,2,6,0,0,1,0,111.11,10.0,10.0,9.0,9.0,22.22
3,Anuj Rawat,1,20,13,0,0,10,1,0,2,0,153.85,20.0,20.0,13.0,6.5,0.00
4,C Green,1,18,19,1,6,10,2,0,1,0,94.74,18.0,18.0,19.0,19.0,31.58


In [30]:
batter_score = batter_performance_score(batter_info, match_phase= "middle", minimum_balls=20)
batter_score.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,JC Buttler,2,38,35,1,0,2,25.71,108.57,38.0,0.695
1,N Wadhera,1,41,34,0,4,1,32.35,120.59,41.0,0.960
2,R Ashwin,1,30,25,1,1,2,44.00,120.00,30.0,0.406
3,R Parag,1,29,24,0,0,2,29.17,120.83,29.0,0.822
4,SS Iyer,1,26,28,0,2,0,32.14,92.86,26.0,0.542


In [31]:
top_batter = batter_score.sort_values(by= ["score"], ascending= False).reset_index().drop(columns=["index"], axis=1)
top_batter.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,N Wadhera,1,41,34,0,4,1,32.35,120.59,41.0,0.960
1,R Parag,1,29,24,0,0,2,29.17,120.83,29.0,0.822
2,JC Buttler,2,38,35,1,0,2,25.71,108.57,38.0,0.695
3,SS Iyer,1,26,28,0,2,0,32.14,92.86,26.0,0.542
4,R Ashwin,1,30,25,1,1,2,44.00,120.00,30.0,0.406


#### Top Batter:

    1. N Wadhera (Score : 0.960)
    2. R Parag (Score : 0.822)
    3. JC Buttler (Score : 0.695)

### 2nd Innings

In [32]:
batter_info = batter_performance(deliveries,match_phase="middle", innings=2,bowling_team="Chennai Super Kings",venue="MA Chidambaram Stadium", season=['2020/21','2021','2022','2023','2024'])
batter_info.head()

,batter,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,strike_rate,batting_average,runs_per_inning,balls_per_dismissal,balls_per_boundary,dot_percentage
0,A Badoni,1,10,8,0,1,4,3,0,0,0,125.00,10.0,10.0,8.0,8.0,12.50
1,AK Markram,1,13,14,1,5,7,1,0,1,0,92.86,13.0,13.0,14.0,14.0,35.71
2,AR Patel,1,6,4,0,1,2,0,0,1,0,150.00,6.0,6.0,4.0,4.0,25.00
3,Abdul Samad,1,19,17,0,6,9,0,0,1,1,111.76,19.0,19.0,17.0,8.5,35.29
4,Atharva Taide,1,8,12,1,5,6,1,0,0,0,66.67,8.0,8.0,12.0,12.0,41.67


In [33]:
batter_score = batter_performance_score(batter_info, match_phase= "middle", minimum_balls= 20)
batter_score.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,B Sai Sudharsan,1,33,28,1,3,0,28.57,117.86,33.0,0.476
1,DA Miller,2,25,24,2,3,0,41.67,104.17,12.5,0.164
2,H Klaasen,1,20,22,1,0,1,36.36,90.91,20.0,0.240
3,LS Livingstone,1,40,24,1,1,4,33.33,166.67,40.0,0.714
4,MK Pandey,1,19,21,1,0,2,57.14,90.48,19.0,0.006


In [34]:
top_batter = batter_score.sort_values(by= ["score"], ascending= False).reset_index().drop(columns=["index"], axis=1)
top_batter.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,N Pooran,2,66,32,1,5,5,21.88,206.25,66.0,0.942
1,MP Stoinis,2,82,53,1,3,5,18.87,154.72,82.0,0.841
2,LS Livingstone,1,40,24,1,1,4,33.33,166.67,40.0,0.714
3,SM Curran,2,37,27,0,3,0,22.22,137.04,37.0,0.576
4,N Rana,1,38,29,0,5,0,27.59,131.03,38.0,0.572


#### Top Batter:

    1. N Pooran (Score : 0.960)
    2. MP Stoinis (Score : 0.841)
    3. LS Livingstone (Score : 0.714)

### Overall

In [36]:
batter_info = batter_performance(deliveries,match_phase="middle",bowling_team="Chennai Super Kings",venue="MA Chidambaram Stadium", season=['2020/21','2021','2022','2023','2024'])
batter_info.head()

,batter,innings,runs,balls,dismissals,dots,ones,twos,threes,fours,sixes,strike_rate,batting_average,runs_per_inning,balls_per_dismissal,balls_per_boundary,dot_percentage
0,A Badoni,1,10,8,0,1,4,3,0,0,0,125.00,10.0,10.0,8.0,8.0,12.50
1,A Raghuvanshi,1,0,1,1,1,0,0,0,0,0,0.00,0.0,0.0,1.0,1.0,100.00
2,AK Markram,2,25,26,2,8,15,1,0,2,0,96.15,12.5,12.5,13.0,13.0,30.77
3,AR Patel,1,6,4,0,1,2,0,0,1,0,150.00,6.0,6.0,4.0,4.0,25.00
4,Abdul Samad,1,19,17,0,6,9,0,0,1,1,111.76,19.0,19.0,17.0,8.5,35.29


In [38]:
batter_score = batter_performance_score(batter_info, match_phase= "middle", minimum_innings= 2, minimum_balls= 30)
batter_score.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,H Klaasen,2,31,34,1,0,1,29.41,91.18,31.0,0.131
1,JC Buttler,2,38,35,1,0,2,25.71,108.57,38.0,0.234
2,MP Stoinis,2,82,53,1,3,5,18.87,154.72,82.0,0.763
3,N Pooran,2,66,32,1,5,5,21.88,206.25,66.0,0.939
4,RK Singh,2,46,44,0,3,2,43.18,104.55,46.0,0.058


In [39]:
top_batter = batter_score.sort_values(by= ["score"], ascending= False).reset_index().drop(columns=["index"], axis=1)
top_batter.head()

,batter,innings,runs,balls,dismissals,fours,sixes,dot_percentage,strike_rate,batting_average,score
0,N Pooran,2,66,32,1,5,5,21.88,206.25,66.0,0.939
1,MP Stoinis,2,82,53,1,3,5,18.87,154.72,82.0,0.763
2,JC Buttler,2,38,35,1,0,2,25.71,108.57,38.0,0.234
3,H Klaasen,2,31,34,1,0,1,29.41,91.18,31.0,0.131
4,RR Rossouw,2,45,41,2,2,2,36.59,109.76,22.5,0.074


#### Top Batter:

    1. N Pooran (Score : 0.939)
    2. MP Stoinis (Score : 0.763)

### Conclusion

It thus concluded that the batters who performed well against spin and could change their scoring pattern while performing good are good enough in the middle overs at Chepauk against CSK. Retired players such as **Shane Watson** and **AB de Villiers**(Both performed better in `1st Innings`) were highly consistent in those situations due to their aggressive yet adaptive natures. Among the available players, **Nicolas Pooran** and **Marcus Stoinis**(Both of them played even better in `2nd Innings`). They control aggression and play strategically during the match against CSK at Chepauk.